# Text into Data

DS 5001 Text as Data

## Purpose

Ww import a text using the  **Clip, Chunk, and Split pattern**.

Demonstrate how to tokenize a raw text and map an OHCO onto the resulting dataframe of tokens.

This goes beyond what we did last week in the First Foray notebook. We capture the chapter, paragraph, and sentence structure of the text.

## Set Up

## Import libraries

In [3]:
import pandas as pd

### Import Config

In [5]:
import configparser
config = configparser.ConfigParser()
config.read("../../../env.ini")
data_home = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']

In [7]:
data_home, output_dir

('/Users/Samantha/Desktop/MSDS/DS5001/data',
 '/Users/Samantha/Desktop/MSDS/DS5001/output')

In [9]:
text_file = f"{data_home}/gutenberg/pg105.txt"
csv_file  = f"{output_dir}/austen-persuasion.csv" # The file we will create

In [11]:
OHCO = ['chap_num', 'para_num', 'sent_num', 'token_num']

## Import file into a dataframe

In [15]:
LINES = pd.DataFrame(open(text_file, 'r', encoding='utf-8-sig').readlines(), columns=['line_str'])
LINES.index.name = 'line_num'
LINES.line_str = LINES.line_str.str.replace(r'\n+', ' ', regex=True).str.strip()

In [17]:
LINES.sample(20)

line_str
line_num                                                   
8592                                                       
1267      The Musgroves, like their houses, were in a st...
5209      He certainly knew what was right, nor could sh...
5833      The following morning Anne was out with her fr...
3250      but Captain Harville had promised them a visit...
8291      best hope of keeping Sir Walter single by the ...
4656                intervals that he could return to Lyme.
6762      at least.  Is this true?  Did he see you last ...
3813      She endeavoured to be composed, and to be just...
7871      Such a letter was not to be soon recovered fro...
1122      received the improvement of a farm-house eleva...
2081                                                       
2045      Anne suppressed a smile, and listened kindly, ...
6214      comparing herself with Miss Larolles, the inim...
2247                                                       
2066      But they have a great many to provide for; and...
1332      one she was now transplanted into.  With the p...
2370      everywhere; but this intimate footing was not ...
7269      state of recently-improved views, of fresh-for...
6483

## Extract Title 

In [19]:
title = LINES.loc[0].line_str.replace('The Project Gutenberg EBook of ', '')

In [21]:
print(title)

Persuasion, by Jane Austen


## Clip the Cruft

In [23]:
clip_pats = [
    r"\*\*\*\s*START OF (?:THE|THIS) PROJECT",
    r"\*\*\*\s*END OF (?:THE|THIS) PROJECT"
]

In [25]:
pat_a = LINES.line_str.str.match(clip_pats[0])
pat_b = LINES.line_str.str.match(clip_pats[1])

In [27]:
line_a = LINES.loc[pat_a].index[0] + 1
line_b = LINES.loc[pat_b].index[0] - 1

In [29]:
line_a, line_b

(19, 8372)

In [31]:
LINES = LINES.loc[line_a : line_b]

In [33]:
LINES.head(10)

line_str
line_num                                                   
19                                                         
20                                                         
21                                                         
22                                                         
23        Produced by Sharon Partridge and Martin Ward. ...
24                                            by Al Haines.
25                                                         
26                                                         
27                                                         
28

In [35]:
LINES.tail(10)

line_str
line_num                                                   
8363                                                       
8364                                                       
8365                                                       
8366                                                       
8367                                                       
8368                                                       
8369                                                       
8370                                                       
8371      End of the Project Gutenberg EBook of Persuasi...
8372

## Chunk by Chapter

### Find all chapter headers

The regex will depend on the source text. You need to investigate the source text to figure this out.

In [38]:
chap_pat = r"^\s*(?:chapter|letter)\s+\d+"

In [40]:
chap_lines = LINES.line_str.str.match(chap_pat, case=False) # Returns a truth vector

In [42]:
LINES.loc[chap_lines] # Use as filter for dataframe

line_str
line_num            
47         Chapter 1
306        Chapter 2
500        Chapter 3
786        Chapter 4
959        Chapter 5
1297       Chapter 6
1657       Chapter 7
1992       Chapter 8
2346       Chapter 9
2632      Chapter 10
3020      Chapter 11
3314      Chapter 12
3876      Chapter 13
4150      Chapter 14
4409      Chapter 15
4677      Chapter 16
4912      Chapter 17
5248      Chapter 18
5654      Chapter 19
5905      Chapter 20
6265      Chapter 21
6970      Chapter 22
7555      Chapter 23
8208      Chapter 24

### Assign numbers to chapters

In [44]:
LINES.loc[chap_lines, 'chap_num'] = [i+1 for i in range(LINES.loc[chap_lines].shape[0])]

In [46]:
LINES.loc[chap_lines]

line_str  chap_num
line_num                      
47         Chapter 1       1.0
306        Chapter 2       2.0
500        Chapter 3       3.0
786        Chapter 4       4.0
959        Chapter 5       5.0
1297       Chapter 6       6.0
1657       Chapter 7       7.0
1992       Chapter 8       8.0
2346       Chapter 9       9.0
2632      Chapter 10      10.0
3020      Chapter 11      11.0
3314      Chapter 12      12.0
3876      Chapter 13      13.0
4150      Chapter 14      14.0
4409      Chapter 15      15.0
4677      Chapter 16      16.0
4912      Chapter 17      17.0
5248      Chapter 18      18.0
5654      Chapter 19      19.0
5905      Chapter 20      20.0
6265      Chapter 21      21.0
6970      Chapter 22      22.0
7555      Chapter 23      23.0
8208      Chapter 24      24.0

Notice that all lines that are not chapter headers have no chapter number assigned to them.

In [48]:
LINES.sample(10)

line_str  chap_num
line_num                                                             
3147      Harville's sister, and was now mourning her lo...       NaN
5608      This was an opportunity which Anne could not r...       NaN
1740      possible; but what was there for a father to d...       NaN
6396      Next week?  To be sure by next week I may be a...       NaN
2174      insignificant barrier, indeed.  Mrs Musgrove w...       NaN
333       claims as an old acquaintance, an attentive ne...       NaN
3320      south-easterly breeze was bringing in with all...       NaN
935       the better feelings of her calmness; but the g...       NaN
7438      "Oh! no.  The usual character of them has noth...       NaN
4469                                                              NaN

### Forward-fill chapter numbers to following text lines

`ffill()` will replace null values with the previous non-null value.

In [50]:
LINES.chap_num = LINES.chap_num.ffill()

In [52]:
LINES.sample(10)

line_str  chap_num
line_num                                                             
1764      is as bad as any of them.  Very unfeeling!  I ...       7.0
1718      enquiries; and Mr Musgrove, no longer under th...       7.0
6646      you ought to have proof, for what is all this ...      21.0
1691      Her brother's return was the first comfort; he...       7.0
1719      his heir, could add his confirmation and prais...       7.0
7735      "Not quite, a few lines more.  I shall have do...      23.0
742                 many of our nobility become so common."       3.0
6950      woman as you, it was not absolutely hopeless. ...      21.0
6208      which was favourable for her.  Colonel Wallis ...      20.0
3281      broken heart, or a mind destroyed by wretchedn...      11.0

Notice that the lines taht precede our first chapter have no chapters, which is what we want. We need to decide whether to keep these lines as textual front matter or to dispose of them.

In [54]:
LINES.head(20)

line_str  chap_num
line_num                                                             
19                                                                NaN
20                                                                NaN
21                                                                NaN
22                                                                NaN
23        Produced by Sharon Partridge and Martin Ward. ...       NaN
24                                            by Al Haines.       NaN
25                                                                NaN
26                                                                NaN
27                                                                NaN
28                                                                NaN
29                                                                NaN
30                                                                NaN
31                                                                NaN
32                                                                NaN
33                                                                NaN
34                                                                NaN
35                                               Persuasion       NaN
36                                                                NaN
37                                                                NaN
38                                                       by       NaN

### Clean up

In [56]:
LINES = LINES.dropna(subset=['chap_num']) # Remove everything before Chapter 1
# LINES = LINES.loc[~LINES.chap_num.isna()] # Remove everything before Chapter 1 (alternate method)
LINES = LINES.loc[~chap_lines] # Remove chapter heading lines; their work is done
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int

In [58]:
LINES.sample(10)

line_str  chap_num
line_num                                                             
2644      some.  Charles Hayter seemed aware of being sl...        10
4669      beginning to be heard at a distance telling th...        15
4945                                 excluded from society.        17
5815                                                               19
5398      odd-looking men walking about here, who, I am ...        18
1122      received the improvement of a farm-house eleva...         5
3806      street; but his evident surprise and vexation ...        12
8118      I saw you with the very person who had guided ...        23
1558      &c., to have another moment for finishing or r...         6
1871      actuated, perhaps, by the same view of escapin...         7

### Group lines into chapters

In [60]:
OHCO[:1]

['chap_num']

In [62]:
# Make big string for each chapter
CHAPS = LINES.groupby(OHCO[:1])\
    .line_str.apply(lambda x: '\n'.join(x))\
    .to_frame('chap_str')

In [64]:
CHAPS.head(10)

chap_str
chap_num                                                   
1         \n\nSir Walter Elliot, of Kellynch Hall, in So...
2         \n\nMr Shepherd, a civil, cautious lawyer, who...
3         \n\n"I must take leave to observe, Sir Walter,...
4         \n\nHe was not Mr Wentworth, the former curate...
5         \n\nOn the morning appointed for Admiral and M...
6         \n\nAnne had not wanted this visit to Uppercro...
7         \n\nA very few days more, and Captain Wentwort...
8         \n\nFrom this time Captain Wentworth and Anne ...
9         \n\nCaptain Wentworth was come to Kellynch as ...
10        \n\nOther opportunities of making her observat...

In [66]:
CHAPS['chap_str'] = CHAPS.chap_str.str.strip()

In [68]:
CHAPS

chap_str
chap_num                                                   
1         Sir Walter Elliot, of Kellynch Hall, in Somers...
2         Mr Shepherd, a civil, cautious lawyer, who, wh...
3         "I must take leave to observe, Sir Walter," sa...
4         He was not Mr Wentworth, the former curate of ...
5         On the morning appointed for Admiral and Mrs C...
6         Anne had not wanted this visit to Uppercross, ...
7         A very few days more, and Captain Wentworth wa...
8         From this time Captain Wentworth and Anne Elli...
9         Captain Wentworth was come to Kellynch as to a...
10        Other opportunities of making her observations...
11        The time now approached for Lady Russell's ret...
12        Anne and Henrietta, finding themselves the ear...
13        The remainder of Anne's time at Uppercross, co...
14        Though Charles and Mary had remained at Lyme m...
15        Sir Walter had taken a very good house in Camd...
16        There was one point which Anne, on returning t...
17        While Sir Walter and Elizabeth were assiduousl...
18        It was the beginning of February; and Anne, ha...
19        While Admiral Croft was taking this walk with ...
20        Sir Walter, his two daughters, and Mrs Clay, w...
21        Anne recollected with pleasure the next mornin...
22        Anne went home to think over all that she had ...
23        One day only had passed since Anne's conversat...
24        Who can be in doubt of what followed?  When an...

So, now we have our text grouped by chapters.

## Split chapters into paragraphs 

We use Pandas' convenient `.split()` method with `expand=True`, followed by `.stack()`.
Note that this creates zero-based indexes.

In [70]:
para_pat = r'\n\n+'

In [34]:
# CHAPS['chap_str'].str.split(para_pat, expand=True).head()

In [72]:
PARAS = CHAPS['chap_str'].str.split(para_pat, expand=True).stack()\
    .to_frame('para_str').sort_index()
PARAS.index.names = OHCO[:2]

In [74]:
PARAS.head()

para_str
chap_num para_num                                                   
1        0         Sir Walter Elliot, of Kellynch Hall, in Somers...
         1                                 "ELLIOT OF KELLYNCH HALL.
         2         "Walter Elliot, born March 1, 1760, married, J...
         3         Precisely such had the paragraph originally st...
         4         Then followed the history and rise of the anci...

In [76]:
PARAS['para_str'] = PARAS['para_str'].str.replace(r'\n', ' ', regex=True)
PARAS['para_str'] = PARAS['para_str'].str.strip()
PARAS = PARAS[~PARAS['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs

In [78]:
PARAS.head()

para_str
chap_num para_num                                                   
1        0         Sir Walter Elliot, of Kellynch Hall, in Somers...
         1                                 "ELLIOT OF KELLYNCH HALL.
         2         "Walter Elliot, born March 1, 1760, married, J...
         3         Precisely such had the paragraph originally st...
         4         Then followed the history and rise of the anci...

## Split paragraphs into sentences

In [80]:
# sent_pat = r'[.?!;:"]+'
sent_pat = r'[.?!;:]+'
SENTS = PARAS['para_str'].str.split(sent_pat, expand=True).stack()\
    .to_frame('sent_str')
SENTS.index.names = OHCO[:3]

In [82]:
SENTS = SENTS[~SENTS['sent_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS.sent_str = SENTS.sent_str.str.strip() # CRUCIAL TO REMOVE BLANK TOKENS

In [84]:
SENTS.head()

sent_str
chap_num para_num sent_num                                                   
1        0        0         Sir Walter Elliot, of Kellynch Hall, in Somers...
                  1         there he found occupation for an idle hour, an...
                  2         there his faculties were roused into admiratio...
                  3         there any unwelcome sensations, arising from d...
                  4         and there, if every other leaf were powerless,...

In [88]:
SENTS.sample(10)

sent_str
chap_num para_num sent_num                                                   
10       23       1                                                         "
17       20       7         Her kind, compassionate visits to this old sch...
9        4        2         Mr Hayter had some property of his own, but it...
4        0        6         They were gradually acquainted, and when acqua...
6        14       8                                             Lord bless me
2        2        7         She wanted more vigorous measures, a more comp...
20       25       2         You have only knowledge enough of the language...
1        16       1         and Elizabeth found him extremely agreeable, a...
22       34       6                                               I recollect
16       0        2         On going down to breakfast the next morning, s...

## Split sentences into tokens

In [90]:
token_pat = r"[\s',-]+"
TOKENS = SENTS['sent_str'].str.split(token_pat, expand=True).stack()\
    .to_frame('token_str')

In [92]:
TOKENS.index.names = OHCO[:4]

In [94]:
TOKENS

token_str
chap_num para_num sent_num token_num            
1        0        0        0                 Sir
                           1              Walter
                           2              Elliot
                           3                  of
                           4            Kellynch
...                                          ...
24       13       0        6                  of
                           7          Persuasion
                           8                  by
                           9                Jane
                           10             Austen

[85014 rows x 1 columns]

## Extract Vocabulary

In [97]:
TOKENS['term_str'] = TOKENS.token_str.replace(r'[\W_]+', '', regex=True).str.lower()
VOCAB = TOKENS.term_str.value_counts().to_frame('n').reset_index().rename(columns={'index':'term_str'})
VOCAB.index.name = 'term_id'

In [99]:
VOCAB

term_str     n
term_id                   
0                the  3330
1                 to  2808
2                and  2800
3                 of  2572
4                  a  1595
...              ...   ...
5755           reins     1
5756     judiciously     1
5757             rut     1
5758            dung     1
5759          austen     1

[5760 rows x 2 columns]

## Gathering by Content Object

In [101]:
def gather(ohco_level):
    global TOKENS
    level_name = OHCO[ohco_level-1].split('_')[0]
    df = TOKENS.groupby(OHCO[:ohco_level])\
        .token_str.apply(lambda x: x.str.cat(sep=' '))\
        .to_frame(f"{level_name}_str")
    return df

In [103]:
gather(1)

chap_str
chap_num                                                   
1         Sir Walter Elliot of Kellynch Hall in Somerset...
2         Mr Shepherd a civil cautious lawyer who whatev...
3         "I must take leave to observe Sir Walter " sai...
4         He was not Mr Wentworth the former curate of M...
5         On the morning appointed for Admiral and Mrs C...
6         Anne had not wanted this visit to Uppercross t...
7         A very few days more and Captain Wentworth was...
8         From this time Captain Wentworth and Anne Elli...
9         Captain Wentworth was come to Kellynch as to a...
10        Other opportunities of making her observations...
11        The time now approached for Lady Russell s ret...
12        Anne and Henrietta finding themselves the earl...
13        The remainder of Anne s time at Uppercross com...
14        Though Charles and Mary had remained at Lyme m...
15        Sir Walter had taken a very good house in Camd...
16        There was one point which Anne on returning to...
17        While Sir Walter and Elizabeth were assiduousl...
18        It was the beginning of February and Anne havi...
19        While Admiral Croft was taking this walk with ...
20        Sir Walter his two daughters and Mrs Clay were...
21        Anne recollected with pleasure the next mornin...
22        Anne went home to think over all that she had ...
23        One day only had passed since Anne s conversat...
24        Who can be in doubt of what followed When any ...

In [105]:
gather(2)

para_str
chap_num para_num                                                   
1        0         Sir Walter Elliot of Kellynch Hall in Somerset...
         1                                  "ELLIOT OF KELLYNCH HALL
         2         "Walter Elliot born March 1 1760 married July ...
         3         Precisely such had the paragraph originally st...
         4         Then followed the history and rise of the anci...
...                                                              ...
24       9         Anne satisfied at a very early period of Lady ...
         10        Her recent good offices by Anne had been enoug...
         11        Mrs Smith s enjoyments were not spoiled by thi...
         12                                                    Finis
         13        End of the Project Gutenberg EBook of Persuasi...

[1008 rows x 1 columns]

In [107]:
gather(3)

sent_str
chap_num para_num sent_num                                                   
1        0        0         Sir Walter Elliot of Kellynch Hall in Somerset...
                  1         there he found occupation for an idle hour and...
                  2         there his faculties were roused into admiratio...
                  3         there any unwelcome sensations arising from do...
                  4         and there if every other leaf were powerless h...
...                                                                       ...
24       11       4         Anne was tenderness itself and she had the ful...
                  5         His profession was all that could ever make he...
                  6         She gloried in being a sailor s wife but she m...
         12       0                                                     Finis
         13       0         End of the Project Gutenberg EBook of Persuasi...

[5612 rows x 1 columns]

## Save work to CSV

This is important -- will be used for homework.

In [109]:
TOKENS.to_csv(csv_file)